[Last.fm ](https://www.last.fm/login?next=/api/account/create)

* API: 912b321d8a267e7892932b6deccb3d2c

Notes for how it would work:
* Users need to create a Last.fm account and connect to Spotify 

In [1]:
!pip install pylast
!pip install requests-toolbelt
!pip install anvil-uplink

import pylast
import sys
import pandas as pd
import requests
import pprint
from requests_toolbelt.threaded import pool

     |████████████████████████████████| 84 kB 951 kB/s 
     |████████████████████████████████| 68 kB 2.6 MB/s 
     |████████████████████████████████| 54 kB 1.1 MB/s 
     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 54 kB 1.8 MB/s 
     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 51 kB 178 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=cf8bcfe462a6c2a8d790f8b172c8565cb84b75184f753f8da4b1f738f3b3116c
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [7]:
import pylast
import sys
import pandas as pd
import requests
import pprint
from requests_toolbelt.threaded import pool

import anvil.server
from tables import app_tables

LASTFM_API_KEY = '912b321d8a267e7892932b6deccb3d2c'
#LASTFM_USER_NAME = 'metaversegods'
TEXT = '#text'
ESTIMATED_TIME_FOR_PROCESSING_PAGE = 352
ESTIMATED_TIME_FOR_PROCESSING_DATAFRAME_PER_PAGE_OF_RESULTS = 275
anvil.server.connect("PDXXW3H2G4I5G3H23UHPXT5N-5XLB6PIE4RL7PMCQ")

@anvil.server.callable
def get_scrobbles(LASTFM_USER_NAME):
  endpoint = "recenttracks"
  username = LASTFM_USER_NAME
  key = '912b321d8a267e7892932b6deccb3d2c'
  limit = 200
  extended = 0
  page = 1
  pages = 0

  """
  endpoint: API endpoint.
  username: Last.fm username to fetch scrobbles for.
  key: API key.
  limit: The number of records per page. Maximum is 200.
  extended: Extended results from API, such as whether user has "liked" the track.
  page: First page to retrieve.
  pages: How many pages of results after "page" argument to retrieve. If 0, get all pages.
  """
  # initialize URL and lists to contain response fields
  url = (
      "https://ws.audioscrobbler.com/2.0/?method=user.get{}"
      "&user={}"
      "&api_key={}"
      "&limit={}"
      "&extended={}"
      "&page={}"
      "&format=json")

  # get total number of pages
  request_url = url.format(endpoint, username, key, limit, extended, page)
  response = requests.get(request_url).json()
  total_pages = int(response[endpoint]["@attr"]["totalPages"])
  if pages > 0:
      total_pages = min([total_pages, pages])

  print(
      "Total pages to retrieve: {}. Estimated time: {}".format(
          total_pages, get_time_remaining(total_pages)
      )
  )

  artist_names = []
  album_names = []
  track_names = []
  timestamps = []
  urls = []

  # add formatted URLs to list to be requested in thread pool
  for page in range(0, int(total_pages) + 1, 1):
      urls.append(url.format(endpoint, username, key, limit, extended, page))
  p = pool.Pool.from_urls(urls)
  p.join_all()

  for response in p.responses():
      if endpoint in response.json():
          response_json = response.json()[endpoint]["track"]
          for track in response_json:
              if "@attr" not in track:
                  artist_names.append(track["artist"][TEXT])
                  album_names.append(track["album"][TEXT])
                  track_names.append(track["name"])
                  timestamps.append(track["date"]["uts"])

  # create and populate a dataframe to contain the data
  df = pd.DataFrame()
  df["artist"] = artist_names
  df["album"] = album_names
  df["track"] = track_names
  df["timestamps"] = timestamps

  df["datetime"] = pd.to_datetime(timestamps, unit="s")
  df["username"] = username

  df.sort_values("timestamps", ascending=False, inplace=True)

  list1 = df[["artist","username"]]
  list1["playcount"] = 1
  list1 = list1.groupby(["artist","username"]).count()
  list1 = list1.sort_values("playcount",ascending=False)
  list1 = list1.reset_index()

  for d in list1.to_dict(orient="records"):
      app_tables.tablee.add_row(**d)


def get_time_remaining(pages_remaining):
    """Calculate the estimated time remaining."""
    millis_remaining = int(
        (pages_remaining * ESTIMATED_TIME_FOR_PROCESSING_PAGE)
        + (
            pages_remaining
            * ESTIMATED_TIME_FOR_PROCESSING_DATAFRAME_PER_PAGE_OF_RESULTS
        )
    )
    seconds_remaining = (millis_remaining / 1000) % 60
    seconds_remaining = int(seconds_remaining)
    minutes_remaining = (millis_remaining / (1000 * 60)) % 60
    minutes_remaining = int(minutes_remaining)
    return "{}m{:2}s".format(minutes_remaining, seconds_remaining)

Total pages to retrieve: 3. Estimated time: 0m 1s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
